In [1]:
from torchtext import vocab, data    
PATH='./'  
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [15]:
import torch
from torch import nn
from fastai.text import *

In [2]:
%ls {PATH}trn

nietzsche.txt        nietzsche_train.txt


In [3]:
TEXT = data.Field(lower=True, tokenize=list)
bs = 64
bptt = 8
n_fac = 42
n_hidden = 256

In [4]:
from fastai.text import *

In [5]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = TextLMDataBunch.from_folder('.', train='trn', valid='val')

In [6]:
import pandas as pd 
x,y = next(iter(md.train_dl))
example = x[:15,:15]
texts = pd.DataFrame([md.train_ds.vocab.textify(l).split(' ') for l in example])
texts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,xxbos,how,xxunk,which,;,thrills,of,--,talent,.,associate,philosophers,so,we,and
1,xxfld,could,minds,now,people,for,will,to,:,xxmaj,the,",",that,hold,we
2,1,you,--,belongs,were,the,",",make,one,socrates,xxup,at,the,our,?--
3,xxup,do,namely,to,agreed,refined,in,also,must,\n,first,the,long,grandfathers,it
4,preface,\n,",",our,in,",",which,the,also,himself,of,best,spun,in,really
5,\n\n\n,xxup,that,unconquerable,the,and,they,bad,have,",",xxmaj,",",-,esteem,seems
6,xxup,differently,a,\n,\n,",",recognized,counter,your,following,europeans,are,out,and,that
7,supposing,?,thought,"""",belief,to,their,-,\n,",",\n,themselves,\n,also,xxup
8,that,xxmaj,comes,flesh,that,sum,own,xxunk,permission,of,according,but,comedy,at,we
9,xxmaj,why,when,and,the,up,strength,against,to,course,to,scholars,of,a,\n


In [7]:
iter_dl = iter(md.train_dl)
for _ in range(5):
    x,y = next(iter_dl)
    print(x.size(), y.size())

torch.Size([69, 64]) torch.Size([69, 64])
torch.Size([65, 64]) torch.Size([65, 64])
torch.Size([75, 64]) torch.Size([75, 64])
torch.Size([66, 64]) torch.Size([66, 64])
torch.Size([66, 64]) torch.Size([66, 64])


In [34]:
len(md.vocab.itos)

5196

In [25]:
class CharSeqStatefulRNN(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0]

In [10]:
class CharSeqStatefulRNN(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
            
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp, h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=1).view(-1, self.vocab_size)

                                                           
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))
            

In [ ]:
rnn1 = CharSeqStatefulRNN()